In [ ]:
import py3Dmol
from Bio.PDB import PDBParser
import pandas as pd
import numpy as np

In [ ]:
file_path = '/home/yarrow/projects/bindcraft-analysis-app/examples/PDL1/final_design_stats.csv'
df = pd.read_csv(file_path)

In [ ]:
df = df.dropna(axis=1, how='all')

#Define metric categories 
af2_metrics_root = ['pLDDT', 'pTM', 'pAE', 'RMSD']
rosetta_metrics_root = ['SASA', 'ShapeComplementarity', 'dG', 'PackStat']
af2_metrics = [col for col in df.columns.tolist() if any(m in col for m in af2_metrics_root)]
rosetta_metrics = [col for col in df.columns.tolist() if any(m in col for m in rosetta_metrics_root)]
other_metrics = [col for col in df.columns.tolist() if col not in af2_metrics or col not in rosetta_metrics]

df_subset = df[af2_metrics + rosetta_metrics]

In [ ]:
for i in df_subset.columns:
    if i[0] == '1':
        df[f'{i[2:]}-ratio'] = np.abs(df[f'1_{i[2:]}'] / df[f'2_{i[2:]}']).round(decimals=2)

In [ ]:

top_metrics = [
'Average_dSASA',
'Average_ShapeComplementarity',
'Average_dG',
'Average_i_pLDDT',
'Average_i_pAE',
'Average_Binder_pLDDT',
'Average_pLDDT',
'Average_pAE',
'Average_pTM',
'Average_Hotspot_RMSD',
'Average_PackStat',
'Average_Relaxed_Clashes',
'Average_n_InterfaceHbonds',
]
    
    

In [39]:
df[top_metrics]

KeyError: "['Average_pTMAverage_Hotspot_RMSD'] not in index"

In [38]:
[i for i in df_subset.columns.tolist() if 'ratio' in i]

['pLDDT-ratio',
 'pTM-ratio',
 'i_pTM-ratio',
 'pAE-ratio',
 'i_pAE-ratio',
 'i_pLDDT-ratio',
 'ss_pLDDT-ratio',
 'Hotspot_RMSD-ratio',
 'Target_RMSD-ratio',
 'Binder_pLDDT-ratio',
 'Binder_pTM-ratio',
 'Binder_pAE-ratio',
 'Binder_RMSD-ratio',
 'ShapeComplementarity-ratio',
 'PackStat-ratio',
 'dG-ratio',
 'dSASA-ratio',
 'dG/dSASA-ratio',
 'Interface_SASA_%-ratio']

In [ ]:
df[af2_metrics + rosetta_metrics]

In [ ]:
df.columns.tolist()

In [ ]:
df.columns.tolist()
plddt = [i for i in df.columns if 'pLDDT' in i and 'Average' not in i]
dfsub = df[['Rank', 'Design']+plddt]


In [ ]:
dfsub

In [ ]:
df.columns.tolist()
pae = [i for i in df.columns if 'pAE' in i and 'Average' not in i]
dfsub2 = df[['Rank', 'Design']+pae]

In [ ]:
dfsub2.head()

In [ ]:
def get_interface_residues(pdb_file, distance_threshold=3.5)->tuple[set, set]:
    """Returns:
        Tuple of two sets:
            - binder residues: set of ('A', resnum)
            - target residues: set of ('B', resnum)
    """
    parser = PDBParser(QUIET=True)
    
    pdb_text = pdb_file.read().decode("utf-8")
    #pdb_io = StringIO(pdb_text) #Make the string a file-like object
    #pdb_io.seek(0)  # Reset file pointer to the beginning 
    
    structure = parser.get_structure('complex', pdb_text)

    model = structure[0]
    Binder = model['A']
    Target = model['B']

    interface_residues_binder = set()
    interface_residues_target = set()

    for residueA in Binder:
        if residueA.id[0] != ' ':  # Skip hetero residues
            continue
        for atomA in residueA:
            for residueB in Target:
                if residueB.id[0] != ' ':  # Skip hetero residues
                    continue
                for atomB in residueB:
                    distance = atomA - atomB # Euclidean distance in Å
                    if distance < distance_threshold:
                        interface_residues_binder.add(('A', residueA.id[1]))
                        interface_residues_target.add(('B', residueB.id[1]))

    return interface_residues_binder, interface_residues_target

In [ ]:
pdb_path = '/home/yarrow/projects/bindcraft-analysis-app/examples/test/Accepted/PDL1-Binder_l102_s533999_mpnn18_model2.pdb'
binders, target = get_interface_residues(pdb_path, distance_threshold=3.5)

In [ ]:
test

In [ ]:
import py3Dmol
def load_pdb(pdb_file):

    pdb_text = open(pdb_file, 'r').read()
    view = py3Dmol.view(width=800, height=400)
    view.addModel(pdb_text, 'pdb')
    view.setStyle({'cartoon': {'color': 'spectrum'}})
    view.setBackgroundColor('0xeeeeee')
    view.zoomTo()
    return view.show()

In [ ]:
import streamlit as st
import pandas as pd
#from binder.parse_bindcraft import load_run

st.set_page_config(layout="wide", page_title="BindCraft Dashboard (Prototype)")

st.sidebar.header("Data")
uploaded_pdb = st.sidebar.file_uploader("Upload PDB)", type=["pdb"])
pdb_name = os.path.basename(uploaded_pdb.name) if uploaded_pdb else None

if uploaded_pdb:
    pdbs = load_pdb(uploaded_pdb)  
else:
    st.info("Upload PDB file to visualize")
    

st.title(f"BindCraft - {pdb_name} Run  Overview")
#st.metric("Designs", len(df))
#st.plotly_chart(...)  # histogram
# Designs table
#selected = st.dataframe(df.sort_values("total_score").head(200))